In [57]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

from pyspark.sql.functions import col, sum, round, countDistinct, max, variance

from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.clustering import KMeans



In [58]:
# Initialize Spark Session
spark = SparkSession.builder.appName("HealthCareModel").getOrCreate()

In [59]:
part_d_data_path = "../DataSet/Prescribers - by Provider and Drug/MUP_DPR_RY23_P04_V10_DY21_NPIBN.csv"
df = spark.read.csv(part_d_data_path, header=True, inferSchema=True)



In [60]:
df.printSchema()


root
 |-- Prscrbr_NPI: integer (nullable = true)
 |-- Prscrbr_Last_Org_Name: string (nullable = true)
 |-- Prscrbr_First_Name: string (nullable = true)
 |-- Prscrbr_City: string (nullable = true)
 |-- Prscrbr_State_Abrvtn: string (nullable = true)
 |-- Prscrbr_State_FIPS: string (nullable = true)
 |-- Prscrbr_Type: string (nullable = true)
 |-- Prscrbr_Type_Src: string (nullable = true)
 |-- Brnd_Name: string (nullable = true)
 |-- Gnrc_Name: string (nullable = true)
 |-- Tot_Clms: integer (nullable = true)
 |-- Tot_30day_Fills: double (nullable = true)
 |-- Tot_Day_Suply: integer (nullable = true)
 |-- Tot_Drug_Cst: double (nullable = true)
 |-- Tot_Benes: integer (nullable = true)
 |-- GE65_Sprsn_Flag: string (nullable = true)
 |-- GE65_Tot_Clms: integer (nullable = true)
 |-- GE65_Tot_30day_Fills: double (nullable = true)
 |-- GE65_Tot_Drug_Cst: double (nullable = true)
 |-- GE65_Tot_Day_Suply: integer (nullable = true)
 |-- GE65_Bene_Sprsn_Flag: string (nullable = true)
 |-- GE65_T

In [61]:
part_d_data_t = df.select(
    col("Prscrbr_NPI").alias("npi"),
    col("Prscrbr_City").alias("city"),
    col("Prscrbr_State_Abrvtn").alias("state"),
    col("Prscrbr_Last_Org_Name").alias("last_name"),
    col("Prscrbr_First_Name").alias("first_name"),
    col("Prscrbr_Type").alias("specialty"),
    col("Brnd_Name").alias("drug_name"),
    col("Gnrc_Name").alias("generic_name"),
    col("Tot_Drug_Cst").alias("total_drug_cost"),
    col("Tot_Clms").alias("total_claim_count"),
    col("Tot_Day_Suply").alias("total_day_supply")
)

In [63]:
# Step 1: Assign one DataFrame to another
part_d_pd1 = part_d_data_t

# Step 2: Select specific columns
part_d_drug_df = part_d_data_t.select("npi", "drug_name", "total_drug_cost", "total_claim_count", "total_day_supply", "specialty")

# Step 3: Change the data type of 'npi' to StringType
part_d_drug_df = part_d_drug_df.withColumn("npi", col("npi").cast(StringType()))

# Step 4: Select specific columns from another DataFrame
part_d_spec_df1 = part_d_data_t.select("npi", "specialty")

# Step 5: Show the first few rows (equivalent to head(0) in Pandas)
part_d_spec_df1.show()
part_d_drug_df.show()

+----------+-----------------+
|       npi|        specialty|
+----------+-----------------+
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
|1003000126|Internal Medicine|
+----------+-----------------+
only showing top 20 rows

+----------+--------------------+---------------+-----------------+----------------+-----------------+
|       npi|           drug_name|total_drug_cost|total_claim_count|total_day_supply|        specialty|
+----------+------------

In [64]:
part_d_pd2 = part_d_data_t.select('npi',
                                  'city',
                                  'state',
                                  'last_name',
                                  'first_name',
                                  'specialty')

part_d_pd2.show()

+----------+--------+-----+---------+----------+-----------------+
|       npi|    city|state|last_name|first_name|        specialty|
+----------+--------+-----+---------+----------+-----------------+
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medicine|
|1003000126|Bethesda|   MD|Enkeshafi|   Ardalan|Internal Medic

In [65]:
part_d_pd_u = part_d_pd2.dropDuplicates()

# Show the result
part_d_pd_u.show()

+----------+-------------+-----+---------+----------+-------------------+
|       npi|         city|state|last_name|first_name|          specialty|
+----------+-------------+-----+---------+----------+-------------------+
|1003028002|      Durango|   CO|   Haynes|      Kent|            Dentist|
|1003041476|     Oak Park|   IL| Lindgren|     Kevin|Allergy/ Immunology|
|1003043555|   Washington|   PA|  Orlosky|     Julie|    Family Practice|
|1003046939|      Spencer|   IA|  Heckert|     Kathi| Nurse Practitioner|
|1003055781|     Hartford|   CT|     Rice|     Jenny|Physician Assistant|
|1003064825|        Parma|   OH| Phillips|    Cherie|   Vascular Surgery|
|1003091539|    Fullerton|   CA|      Woo|      Kiho|  Internal Medicine|
|1003095167|      El Paso|   IL|    Tyner|Jean-Marie| Nurse Practitioner|
|1003101932|   Cumberland|   MD|     Hong|     Feiyu|  Internal Medicine|
|1003119009|     Bellevue|   WA|  Swenson|   Jessica| Nurse Practitioner|
|1003127002|       Laredo|   TX|  Rami

In [66]:
group_cols = ['npi']

part_d_pd3 = (part_d_pd1.groupBy(group_cols)
             .agg(
                 F.sum("total_drug_cost").alias("sum_total_drug_cost"),
                 F.mean("total_drug_cost").alias("mean_total_drug_cost"),
                 F.max("total_drug_cost").alias("max_total_drug_cost"),
                 F.sum("total_claim_count").alias("sum_total_claim_count"),
                 F.mean("total_claim_count").alias("mean_total_claim_count"),
                 F.max("total_claim_count").alias("max_total_claim_count"),
                 F.sum("total_day_supply").alias("sum_total_day_supply"),
                 F.mean("total_day_supply").alias("mean_total_day_supply"),
                 F.max("total_day_supply").alias("max_total_day_supply")
             ))

# Cast to float if necessary (optional)
for col_name in part_d_pd3.columns:
    part_d_pd3 = part_d_pd3.withColumn(col_name, F.col(col_name).cast("float"))

# Show the result
part_d_pd3.show()

+------------+-------------------+--------------------+-------------------+---------------------+----------------------+---------------------+--------------------+---------------------+--------------------+
|         npi|sum_total_drug_cost|mean_total_drug_cost|max_total_drug_cost|sum_total_claim_count|mean_total_claim_count|max_total_claim_count|sum_total_day_supply|mean_total_day_supply|max_total_day_supply|
+------------+-------------------+--------------------+-------------------+---------------------+----------------------+---------------------+--------------------+---------------------+--------------------+
| 1.0030432E9|           18613.12|             930.656|            7077.18|                355.0|                 17.75|                 44.0|             21040.0|               1052.0|              3000.0|
|1.00307283E9|          282157.47|           3399.4878|           45438.26|               4100.0|              49.39759|                244.0|            203448.0|         

In [67]:
part_d_pd3.count()

1017417

In [69]:
part_d_all_pd = part_d_pd3.join(part_d_pd_u, on='npi', how='left')

# Show the result
part_d_all_pd.show()

+------------+-------------------+--------------------+-------------------+---------------------+----------------------+---------------------+--------------------+---------------------+--------------------+--------------+-----+--------------+----------+--------------------+
|         npi|sum_total_drug_cost|mean_total_drug_cost|max_total_drug_cost|sum_total_claim_count|mean_total_claim_count|max_total_claim_count|sum_total_day_supply|mean_total_day_supply|max_total_day_supply|          city|state|     last_name|first_name|           specialty|
+------------+-------------------+--------------------+-------------------+---------------------+----------------------+---------------------+--------------------+---------------------+--------------------+--------------+-----+--------------+----------+--------------------+
| 1.0030124E9|             577.08|              288.54|             373.53|                 70.0|                  35.0|                 39.0|               478.0|            

In [75]:
payment_data_path="../DataSet/PaymentDataSet/OP_DTL_OWNRSHP_PGYR2021_P06302023.csv"
pds = spark.read.csv(payment_data_path, header=True, inferSchema=True)

In [76]:
pds.printSchema()

root
 |-- Change_Type: string (nullable = true)
 |-- Physician_Profile_ID: integer (nullable = true)
 |-- Physician_NPI: integer (nullable = true)
 |-- Physician_First_Name: string (nullable = true)
 |-- Physician_Middle_Name: string (nullable = true)
 |-- Physician_Last_Name: string (nullable = true)
 |-- Physician_Name_Suffix: string (nullable = true)
 |-- Recipient_Primary_Business_Street_Address_Line1: string (nullable = true)
 |-- Recipient_Primary_Business_Street_Address_Line2: string (nullable = true)
 |-- Recipient_City: string (nullable = true)
 |-- Recipient_State: string (nullable = true)
 |-- Recipient_Zip_Code: string (nullable = true)
 |-- Recipient_Country: string (nullable = true)
 |-- Recipient_Province: string (nullable = true)
 |-- Recipient_Postal_Code: string (nullable = true)
 |-- Physician_Primary_Type: string (nullable = true)
 |-- Physician_Specialty: string (nullable = true)
 |-- Record_ID: integer (nullable = true)
 |-- Program_Year: integer (nullable = true)

In [78]:
pds_df = pds.select(
    col("Physician_First_Name").alias("first_name"),
    col("Physician_Last_Name").alias("last_name"),
    col("Recipient_State").alias("state"),
    col("Recipient_City").alias("city"),
    col("Total_Amount_Invested_USDollars").alias("total_amount_of_payment_usd")
)

pds_df.show()

+----------+----------+-----+----------------+---------------------------+
|first_name| last_name|state|            city|total_amount_of_payment_usd|
+----------+----------+-----+----------------+---------------------------+
|     Faith|    Brosch|   DE|          NEWARK|                        0.0|
|      Troy|  Brothers|   FL|          Naples|                        0.0|
|      Anne|     Brown|   VA|        Leesburg|                        0.0|
|     David|     Brown|   FL|      FORT MYERS|                        0.0|
|    Ingrid|     Brown|   TX|      Round Rock|                        0.0|
|     Kevin|     Brown|   TX|      Round Rock|                        0.0|
|    Steven|     Brown|   FL|      Palm Coast|                        0.0|
|   Michael|   Buckley|   NC|         Raleigh|                        0.0|
|  Christin|Richardson|   NC|         Raleigh|                        0.0|
|     Kathy|Richardson|   NC|      Greensboro|                        0.0|
|    Kellyn|    Rielly|  

In [82]:
pds_df1 = (pds_df
           .groupBy('first_name', 'last_name', 'state', 'city')
           .agg(F.sum('total_amount_of_payment_usd').alias('sum_total_amount_of_payment_usd'))
           .withColumn('total_amount_of_payment_usd', F.col('sum_total_amount_of_payment_usd').cast('float')))

# Show the result
pds_df1.show()

+-----------+----------+-----+---------------+-------------------------------+---------------------------+
| first_name| last_name|state|           city|sum_total_amount_of_payment_usd|total_amount_of_payment_usd|
+-----------+----------+-----+---------------+-------------------------------+---------------------------+
|       Mary|   Goodwin|   NC|      Asheville|                            0.0|                        0.0|
|Christopher|      Ames|   CA|  SAN FRANCISCO|                            0.0|                        0.0|
|   Rajeshri|     Patel|   NY|     Long Beach|                        38348.0|                    38348.0|
|     George|    Ferzli|   NY|  Staten Island|                        70000.0|                    70000.0|
|    Michael| Ziebelman|   FL|   Winter Haven|                            0.0|                        0.0|
|    STEPHEN|     RAMEE|   LA|    NEW ORLEANS|                            0.0|                        0.0|
|       JOHN|    VUKICH|   WI|      W

In [83]:
# Convert all string columns to uppercase
for col_name in pds_df1.columns:
    # Check if the column type is string
    if isinstance(pds_df1.schema[col_name].dataType, StringType):
        pds_df1 = pds_df1.withColumn(col_name, F.upper(F.col(col_name)))

# Show the result
pds_df1.show()

+-----------+----------+-----+---------------+-------------------------------+---------------------------+
| first_name| last_name|state|           city|sum_total_amount_of_payment_usd|total_amount_of_payment_usd|
+-----------+----------+-----+---------------+-------------------------------+---------------------------+
|       MARY|   GOODWIN|   NC|      ASHEVILLE|                            0.0|                        0.0|
|CHRISTOPHER|      AMES|   CA|  SAN FRANCISCO|                            0.0|                        0.0|
|   RAJESHRI|     PATEL|   NY|     LONG BEACH|                        38348.0|                    38348.0|
|     GEORGE|    FERZLI|   NY|  STATEN ISLAND|                        70000.0|                    70000.0|
|    MICHAEL| ZIEBELMAN|   FL|   WINTER HAVEN|                            0.0|                        0.0|
|    STEPHEN|     RAMEE|   LA|    NEW ORLEANS|                            0.0|                        0.0|
|       JOHN|    VUKICH|   WI|      W

In [85]:
pay_part_d_fpd = part_d_all_pd.join(pds_df1, on=['last_name', 'first_name', 'city', 'state'], how='left')

# Show the result
pay_part_d_fpd.show()

+--------------+----------+--------------+-----+------------+-------------------+--------------------+-------------------+---------------------+----------------------+---------------------+--------------------+---------------------+--------------------+--------------------+-------------------------------+---------------------------+
|     last_name|first_name|          city|state|         npi|sum_total_drug_cost|mean_total_drug_cost|max_total_drug_cost|sum_total_claim_count|mean_total_claim_count|max_total_claim_count|sum_total_day_supply|mean_total_day_supply|max_total_day_supply|           specialty|sum_total_amount_of_payment_usd|total_amount_of_payment_usd|
+--------------+----------+--------------+-----+------------+-------------------+--------------------+-------------------+---------------------+----------------------+---------------------+--------------------+---------------------+--------------------+--------------------+-------------------------------+------------------------

In [ ]:
pay_part_d_fpd.count()